# 05 - Interpretability & Reporting

Explain model predictions and provide insights using SHAP values and other interpretability techniques.

**Sections:**
1. **SHAP Values Analysis**: Global and local feature importance
2. **Feature Impact Analysis**: Which features drive podium predictions
3. **Case Studies**: Analyze specific race predictions
4. **Model Calibration**: Calibration curve analysis
5. **Business Insights**: Key findings and recommendations

**Input:** `models/best_podium_model.pkl`, `data/processed/features.csv`


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report, roc_auc_score, brier_score_loss
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.calibration import calibration_curve

# SHAP for interpretability
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False
    print("Warning: SHAP not available")

import joblib

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


In [ ]:
# Set up paths
# Get project root (works whether running from notebooks/ or F1/ folder)
PROJECT_ROOT = Path().resolve()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent

PROCESSED_ROOT = PROJECT_ROOT / "data" / "processed"
MODELS_ROOT = PROJECT_ROOT / "models"

# Load model
model_path = MODELS_ROOT / "best_podium_model.pkl"
if not model_path.exists():
    # Try to find any model file
    model_files = list(MODELS_ROOT.glob("*.pkl"))
    if model_files:
        model_path = model_files[0]
    else:
        raise FileNotFoundError("No model file found in models/ directory")

model = joblib.load(model_path)
print(f"Loaded model from: {model_path}")

# Load features
features = pd.read_csv(PROCESSED_ROOT / "features.csv")
features['date'] = pd.to_datetime(features['date'], errors='coerce')

# Prepare data
X = features.drop(columns=['podium', 'raceId', 'driverId', 'date', 'year'], errors='ignore')
y = features['podium']

# Handle categorical features (same as in modeling notebook)
from sklearn.preprocessing import LabelEncoder
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str).fillna('Unknown'))

# Fill missing values
for col in X.columns:
    if X[col].dtype in [np.float64, np.int64]:
        X[col] = X[col].fillna(X[col].median())
    else:
        X[col] = X[col].fillna(X[col].mode()[0] if len(X[col].mode()) > 0 else 0)

# Test set (2024)
test_mask = features['year'] == 2024
X_test = X[test_mask]
y_test = y[test_mask]
features_test = features[test_mask].reset_index(drop=True)

print(f"Test set: {len(X_test):,} samples (2024)")


In [ ]:
## 1. Model Performance Summary

Evaluate model performance on test set.


In [ ]:
# Predictions on test set
y_test_pred = model.predict_proba(X_test)[:, 1]
y_test_pred_binary = (y_test_pred >= 0.5).astype(int)

# Metrics
test_metrics = {
    'roc_auc': roc_auc_score(y_test, y_test_pred),
    'brier': brier_score_loss(y_test, y_test_pred)
}

print("Test Set Performance (2024):")
print(f"  ROC-AUC: {test_metrics['roc_auc']:.4f}")
print(f"  Brier Score: {test_metrics['brier']:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_test_pred_binary, target_names=['Non-Podium', 'Podium']))


In [ ]:
## 2. SHAP Values Analysis

Calculate SHAP values for global and local feature importance.


In [ ]:
if SHAP_AVAILABLE:
    # Create SHAP explainer
    # Use a sample of test data for faster computation
    sample_size = min(100, len(X_test))
    X_test_sample = X_test.iloc[:sample_size]
    
    try:
        # Try TreeExplainer first (for LightGBM, CatBoost, XGBoost)
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test_sample)
        
        # Handle binary classification (shap_values might be a list)
        if isinstance(shap_values, list):
            shap_values = shap_values[1]  # Use positive class
        
        print(f"SHAP values calculated for {sample_size} samples")
        
        # Global feature importance
        shap.summary_plot(shap_values, X_test_sample, plot_type="bar", show=False)
        plt.title('Global Feature Importance (SHAP)')
        plt.tight_layout()
        plt.show()
        
        # Detailed summary plot
        shap.summary_plot(shap_values, X_test_sample, show=False)
        plt.title('SHAP Summary Plot')
        plt.tight_layout()
        plt.show()
        
        # Feature importance DataFrame
        feature_importance_shap = pd.DataFrame({
            'feature': X_test_sample.columns,
            'importance': np.abs(shap_values).mean(axis=0)
        }).sort_values('importance', ascending=False)
        
        print("\nTop 15 Features by SHAP Importance:")
        print(feature_importance_shap.head(15).to_string(index=False))
        
    except Exception as e:
        print(f"TreeExplainer failed: {e}")
        print("Trying KernelExplainer (slower but more general)...")
        try:
            # Fallback to KernelExplainer
            explainer = shap.KernelExplainer(model.predict_proba, X_test_sample.iloc[:50])
            shap_values = explainer.shap_values(X_test_sample.iloc[:10])
            
            if isinstance(shap_values, list):
                shap_values = shap_values[1]
            
            shap.summary_plot(shap_values, X_test_sample.iloc[:10], plot_type="bar", show=False)
            plt.title('Global Feature Importance (SHAP - Kernel)')
            plt.tight_layout()
            plt.show()
            
        except Exception as e2:
            print(f"KernelExplainer also failed: {e2}")
            print("SHAP analysis skipped")
            shap_values = None
else:
    print("SHAP not available - skipping SHAP analysis")
    shap_values = None


In [ ]:
## 3. Feature Impact Analysis

Analyze which features drive podium predictions most.


In [ ]:
# Permutation importance as alternative to SHAP
perm_result = permutation_importance(
    model,
    X_test,
    y_test,
    n_repeats=5,
    random_state=42,
    n_jobs=-1,
    scoring='roc_auc'
)
perm_importances = pd.Series(perm_result.importances_mean, index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 8))
perm_importances.head(15).sort_values().plot.barh(color='steelblue')
plt.title('Top 15 Feature Importances (Permutation)')
plt.xlabel('Importance (Decrease in ROC-AUC)')
plt.tight_layout()
plt.show()

print("Top 15 Features by Permutation Importance:")
print(perm_importances.head(15).to_string())


In [ ]:
## 4. Partial Dependence Plots

Visualize how individual features affect predictions.


In [ ]:
# Partial dependence plots for top features
top_features = perm_importances.head(3).index.tolist()
available_features = [f for f in top_features if f in X_test.columns]

if available_features:
    fig, axes = plt.subplots(1, len(available_features), figsize=(5 * len(available_features), 4))
    if len(available_features) == 1:
        axes = [axes]
    
    for feature_name, axis in zip(available_features, axes):
        try:
            PartialDependenceDisplay.from_estimator(
                model,
                X_test,
                features=[feature_name],
                kind='average',
                ax=axis
            )
            axis.set_title(f'Partial Dependence: {feature_name}')
        except Exception as e:
            print(f"Could not create PDP for {feature_name}: {e}")
    
    plt.tight_layout()
    plt.show()
else:
    print("No suitable features found for partial dependence plots")


In [ ]:
## 5. Case Studies

Analyze specific race predictions - high confidence and surprising predictions.


# Find high-confidence predictions
high_confidence = features_test.copy()
high_confidence['predicted_prob'] = y_test_pred
high_confidence['actual'] = y_test
high_confidence['error'] = abs(high_confidence['predicted_prob'] - high_confidence['actual'])

# High confidence correct predictions
high_conf_correct = high_confidence[
    (high_confidence['predicted_prob'] > 0.8) & (high_confidence['actual'] == 1)
].sort_values('predicted_prob', ascending=False)

# High confidence incorrect (surprising)
high_conf_incorrect = high_confidence[
    (high_confidence['predicted_prob'] > 0.7) & (high_confidence['actual'] == 0)
].sort_values('predicted_prob', ascending=False)

print("High Confidence Correct Predictions (Top 5):")
if len(high_conf_correct) > 0:
    display_cols = ['surname', 'circuit_name', 'grid', 'predicted_prob', 'actual']
    available_cols = [c for c in display_cols if c in high_conf_correct.columns]
    print(high_conf_correct[available_cols].head(5).to_string(index=False))
else:
    print("None found")

print("\nSurprising Predictions (High Confidence but Wrong - Top 5):")
if len(high_conf_incorrect) > 0:
    print(high_conf_incorrect[available_cols].head(5).to_string(index=False))
else:
    print("None found")

# Local SHAP explanations for a few examples
if SHAP_AVAILABLE and shap_values is not None:
    print("\nLocal SHAP Explanations (Sample):")
    # Show SHAP waterfall for first high-confidence prediction
    if len(high_conf_correct) > 0:
        idx = high_conf_correct.index[0]
        if idx < len(X_test_sample):
            shap.waterfall_plot(
                shap.Explanation(
                    values=shap_values[idx],
                    base_values=explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
                    data=X_test_sample.iloc[idx],
                    feature_names=X_test_sample.columns
                ),
                show=False
            )
            plt.title(f'SHAP Explanation for High-Confidence Prediction')
            plt.tight_layout()
            plt.show()


## 6. Model Calibration

Analyze model calibration - how well do predicted probabilities match observed frequencies?


In [ ]:
# Calibration curve
prob_true, prob_pred = calibration_curve(y_test, y_test_pred, n_bins=10)

plt.figure(figsize=(8, 6))
plt.plot(prob_pred, prob_true, marker='o', linewidth=2, label='Model')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly Calibrated')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Observed Frequency')
plt.title('Model Calibration Curve (Test Set 2024)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Brier Score: {brier_score_loss(y_test, y_test_pred):.4f}")
print("(Lower is better - measures calibration)")


## 7. Business Insights Summary

Key findings and recommendations for feature engineering improvements.


In [ ]:
print("Key Insights:")
print("=" * 60)

print("\n1. Top Predictive Features:")
if 'feature_importance_shap' in locals():
    print("   (Based on SHAP values)")
    print(feature_importance_shap.head(10).to_string(index=False))
else:
    print("   (Based on Permutation Importance)")
    print(perm_importances.head(10).to_string())

print("\n2. Model Performance:")
print(f"   Test ROC-AUC: {test_metrics['roc_auc']:.4f}")
print(f"   Test Brier Score: {test_metrics['brier']:.4f}")

print("\n3. Recommendations:")
print("   - Focus on grid position and qualifying performance (strongest predictors)")
print("   - Driver and constructor historical performance are important")
print("   - Consider adding more FastF1 telemetry features (when available)")
print("   - Weather features may improve predictions for 2018+ races")

# Save insights
insights = {
    'top_features': perm_importances.head(20).to_dict(),
    'test_metrics': test_metrics,
    'calibration': {
        'brier_score': float(brier_score_loss(y_test, y_test_pred))
    }
}

import json
insights_path = PROCESSED_ROOT / "interpretability_insights.json"
with open(insights_path, 'w') as f:
    json.dump(insights, f, indent=2, default=str)

print(f"\nInsights saved to: {insights_path}")
